In [36]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.stats
from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D
import operator
from scipy.optimize import minimize
import math

In [37]:
#initialize plotting canvas
%matplotlib tk 
fig = plt.figure( num=1, clear=True)
#seed
seed_gen=np.random.uniform(0,1000,1).astype(int)
np.random.seed(seed=seed_gen)

In [38]:
#generate one path
def gen_path(x_0in,N_in,T_in, b_in, sig_in):
    #T  time limit
    # N #steps
    dt = T_in/N_in 
    x = np.zeros(N_in) #container
    #generate normal distributed 
    dW=np.random.normal(0, 1, N_in)
    #initialize
    # x_0 
    x[0]=x_0in
    for i in range(1,N):
        x[i] =x[i-1] + b_in*dt + sig_in*np.sqrt(dt)*dW[i]
    return(x)

In [39]:
# the liklihood function of the gaussian transition densities
def dist_b(param_b,X,sig_in,T_in): #X are pairs of x_i and x_{i-1}
    f=0
    dt=T_in/len(X)
    sig=10#set sigma
    b=param_b
    for i in range(1,len(X)):
        f=f -(X[i]-X[i-1] - b*dt)**2/(2*dt*(sig)**2) - np.log(np.sqrt(2*math.pi*dt)*sig)
    return(-1*f)

In [40]:
# the liklihood function of the gaussian transition densities
def dist_sig(param_sig,X,b_in,T_in): #X are pairs of x_i and x_{i-1}
    f=0
    dt=T_in/len(X)
    b=b_in # set b
    sig=param_sig
    for i in range(1,len(X)):
        f=f -(X[i]-X[i-1] - b*dt)**2/(2*dt*(sig)**2) - np.log(np.sqrt(2*math.pi*dt)*sig) 
    return(-1*f)

In [41]:
# the liklihood function of the gaussian transition densities
def dist_2d(param_2d,X, T_in): #X are pairs of x_i and x_{i-1}
    f=0
    dt=T_in/len(X)
    sig=param_2d[1]
    b=param_2d[0]
    
    #gamma prior
    
    a=2
    loc=9
    scale=3
    
    for i in range(1,len(X)):
        f=f -(X[i]-X[i-1] - b*dt)**2/(2*dt*(sig)**2) - np.log(np.sqrt(2*math.pi*dt)*sig) 
            #+np.log(scipy.stats.gamma.pdf(sig, a, loc, scale))
    return(-1*f)

In [42]:
a=1.5
loc=8
scale=0.5
scipy.stats.gamma.pdf(8.1, a, loc, scale)

0.8263064759476444

In [43]:
#path set parameters
N=10000
#sig_gen = np.zeros(N);
#b_gen = np.zeros(N);
sig_gen=10
b_gen=100
x_0=0
T=1
dt=T/N

data=gen_path(x_0in=x_0,N_in=N,T_in=T, b_in=b_gen, sig_in=sig_gen)


In [44]:
ax1 = fig.add_subplot(221)
ax1.cla()
ax1.plot(data);

In [45]:
int_mean=np.random.uniform(-1000,1000,1)
T=1
sig_op=sig_gen
args=(data,sig_op,T)
x0=int_mean
min_param=minimize(dist_b,x0,args, tol=1e-15);min_param

      fun: -8741.90964369142
 hess_inv: array([[32.74598835]])
      jac: array([-0.00024414])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 179
      nit: 5
     njev: 56
   status: 2
  success: False
        x: array([76.89430856])

In [46]:
int_sig=np.random.uniform(0,100,1)
T=1
b_op=b_gen
args=(data,b_op ,T)
x0=int_sig
bnds = np.array([(0.001,5000)])
min_param=minimize(dist_sig,x0,args,bounds=bnds, tol=1e-12);min_param

      fun: array([-8740.15613733])
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.0336513])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 6
   status: 0
  success: True
        x: array([10.09588029])

In [47]:
int_param=np.array(  [3,1]  , dtype=float);
T=1
args=(data,T)
x0=int_param
#bnds = ((10, 10), (500, 25))
#min_param=minimize(dist_2d,x0,args,bounds=bnds, tol=1e-12);min_param
min_param=minimize(dist_2d,x0,args, tol=1e-12);min_param

      fun: -8742.776369865423
 hess_inv: array([[ 0.0127695 , -0.00637836],
       [-0.00637836,  0.00326803]])
      jac: array([-0.00427246, -0.01672363])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 619
      nit: 34
     njev: 152
   status: 2
  success: False
        x: array([76.86859668, 10.09324915])

In [48]:
x, y = np.mgrid[-200:200:10, 1:25:1]
pos = np.empty(x.shape + (2,))
pos[:, :, 0] = x; pos[:, :, 1] = y

In [49]:
value_pos=np.empty((pos.shape[0],pos.shape[1] ))
for i in range(0,pos.shape[0]):
    for j in range(0,pos.shape[1]):
        value_pos[i,j]=dist_2d(pos[i,j,:],data,T_in=1)

In [53]:
ax2 = fig.add_subplot(212, projection='3d')
ax2.cla()
ax2.plot_surface(x, y, value_pos, cmap=cm.coolwarm,linewidth=0, antialiased=False)
ax2.scatter(min_param.x[0], min_param.x[1], dist_2d(min_param.x,data,T_in=1), marker='^',s=600)